In [1]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd
import ast
from scipy import stats

experiments = [
    {
        'name': 'random-uniform-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },

    {
        'name': 'random-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'fixed-pixel-variable-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'hsl-force-first-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0)]
        }
    },
    
    {
        'name': 'hsl-force-last-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(99.0, 0.0, 0.0)]
        }
    }
    
    {
        'name': 'blur-big-images',
        'filters': ['blur'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {}
    }
]

command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

trim = 0.25

def runExperiment(filter, arguments):
    comm = command[filter].format(**arguments)
    delta = subprocess.check_output(comm, shell=True)
    delta = ast.literal_eval(delta.strip())
    rdelta = pd.Series(delta)
    
    return {
            'delta': delta,
            'mean': rdelta.mean(),
            'var': rdelta.var(),
            'tmean': stats.trim_mean(rdelta, trim),
            'min': rdelta.min()
        }

# makeparams = ['o3', 'o3fast', 'o3shifts', 'o3fastshifts']
def experimentWorker(experiment, param, dataset, procImages):
    output = []
    
    for (f, l, src) in itertools.product(experiment['filters'], experiment['languages'], procImages):
        expOutput = {
            'experiment': experiment['name'],
            'make-param': param,
            'filename': os.path.splitext(os.path.basename(src))[0],
            'filter': f,
            'language': l
        }
        
        expOutput['size'] = expOutput['filename'].split(' ')[1].split('x')[0]

        arguments = {
            'language': l,
            'filter': f,
            'src': src,
            'dst': '/dev/null'
        }
        
        if f == 'hsl':
            for (h, s, l) in experiment['params'][f]:
                pars = {
                        'h': h,
                        's': s,
                        'l': l
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args.update(pars)
                args.update(expOutput)
                output.append(args)
        elif f == 'merge':
            for alpha in experiment['params'][f]:
                pars = {
                        'alpha': alpha,
                        'src2': src
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args['alpha'] = alpha
                args.update(expOutput)
                output.append(args)
        else:
            args = runExperiment(f, arguments)
            args.update(expOutput)
            output.append(args)

    dataset += output
    del output

manager = multiprocessing.Manager()
dataset = manager.list([])

base_path = '/home/julian/Development/orga2-tp2'
src_path = os.path.join(base_path, 'src')
filters_path = os.path.join(src_path, 'filters')
test_path = os.path.join(src_path, 'tests')
makeparams = ['o3shuffles', 'o3fastshuffles', 'o3shift', 'o3fastshift']

exp_img_path = os.path.join(base_path, 'exp', 'images')

for param in makeparams:
    subprocess.check_output('make -C "' + filters_path + '" clean', executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '" clean' , executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + filters_path + '" ' + param, executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '"' , executable='/bin/bash', shell=True)
    
    jobs = []

    for experiment in experiments:
        images_path = os.path.join(exp_img_path, experiment['name'])

        procImages = []
        
        for img in os.listdir(images_path):
            rpath = os.path.join(images_path, img)

            if os.path.isfile(rpath):
                procImages.append(rpath)

        p = multiprocessing.Process(target=experimentWorker, args=(experiment, param, dataset, procImages))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()

In [3]:
ds = []

for x in dataset:
    ds.append(x)
    
ds = pd.DataFrame(ds)

In [5]:
ds.to_csv('dataset.csv')

In [2]:
import pandas as pd
ds = pd.read_csv('dataset.csv')

In [15]:
%matplotlib inline
ds[(ds['language'] == 'asm2') & (ds['filter'] == 'hsl') & (ds['experiment'] == 'hsl-force-first-if') & (ds['make-param'] == 'o3fast')].sort('size')

,Unnamed: 0,alpha,experiment,filename,filter,h,l,language,make-param,mean,min,s,size,tmean,var
10787,10787,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 4x4 (1804289383),hsl,0,0,asm2,o3fast,4493.29,4421,0,4,4463.44,3.012687e+04
10799,10799,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 8x8 (846930886),hsl,0,0,asm2,o3fast,27191.36,26998,0,8,27159.88,7.941761e+04
10758,10758,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 12x12 (1681692777),hsl,0,0,asm2,o3fast,36431.46,36130,0,12,36420.42,3.633732e+04
10717,10717,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 16x16 (1714636915),hsl,0,0,asm2,o3fast,90723.84,89978,0,16,90249.22,1.994325e+07
10773,10773,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 20x20 (1957747793),hsl,0,0,asm2,o3fast,213795.89,175639,0,20,176075.66,6.725307e+10
10756,10756,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 24x24 (424238335),hsl,0,0,asm2,o3fast,276952.89,252619,0,24,253077.62,5.573802e+10
10700,10700,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 28x28 (719885386),hsl,0,0,asm2,o3fast,220861.68,219940,0,28,220265.12,1.812482e+07
10732,10732,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 32x32 (1649760492),hsl,0,0,asm2,o3fast,797810.76,448898,0,32,449654.32,1.208453e+13
10720,10720,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 36x36 (596516649),hsl,0,0,asm2,o3fast,455937.98,454734,0,36,455214.72,1.252011e+07
10769,10769,NaN,hsl-force-first-if,hsl-empty-if-0-0-0 40x40 (1189641421),hsl,0,0,asm2,o3fast,979240.02,401128,0,40,702364.20,6.709643e+12


In [6]:
ds['experiment'].value_counts()
ds['make-param'].value_counts()

o3fast    5100
o0        5100
o3        5100
dtype: int64